In [1]:
import MySQLdb
import sys


In [2]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [3]:
import bz2

def print_logs_configuration(cid):
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        cursor = socket.cursor()

        query = "SELECT stdout_log_file, stderr_log_file, user_output_file FROM compilations WHERE cid = %d" % cid
        cursor.execute(query)
        stdlog_file, errlog_file, output_file = cursor.fetchone()
            
        if (stdlog_file is None) or (errlog_file is None) or (output_file is None):
            print ("Unable to retrieve cid...")
            return
        
        print("STD LOG FILE")
        print(bz2.decompress(stdlog_file).decode('ascii'))
        print("#########")
        print("ERR LOG FILE")
        print("#########")
        print(bz2.decompress(errlog_file).decode()) # not ascii? 
        print("#########")
        print("OUTPUT LOG FILE")
        print("#########")
        print(bz2.decompress(output_file).decode('ascii'))
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

print_logs_configuration(16732) 

STD LOG FILE
make: Entering directory '/TuxML/linux-4.13.3'
scripts/kconfig/conf  --silentoldconfig Kconfig
  SYSTBL  arch/x86/entry/syscalls/../../include/generated/asm/syscalls_32.h
  SYSHDR  arch/x86/entry/syscalls/../../include/generated/asm/unistd_32_ia32.h
  SYSHDR  arch/x86/entry/syscalls/../../include/generated/asm/unistd_64_x32.h
  SYSTBL  arch/x86/entry/syscalls/../../include/generated/asm/syscalls_64.h
  SYSHDR  arch/x86/entry/syscalls/../../include/generated/uapi/asm/unistd_32.h
  CHK     include/config/kernel.release
  SYSHDR  arch/x86/entry/syscalls/../../include/generated/uapi/asm/unistd_64.h
  UPD     include/config/kernel.release
  SYSHDR  arch/x86/entry/syscalls/../../include/generated/uapi/asm/unistd_x32.h
  CHK     include/generated/uapi/linux/version.h
  UPD     include/generated/uapi/linux/version.h
  CHK     include/generated/utsrelease.h
  UPD     include/generated/utsrelease.h
  HOSTCC  arch/x86/tools/relocs_32.o
  HOSTCC  arch/x86/tools/relocs_64.o
  HOSTCC  a

In [8]:
def get_configuration_file(cid):
    print("Getting configuration file of cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        cursor = socket.cursor()

        query = "SELECT config_file FROM compilations WHERE cid = %d" % cid
        cursor.execute(query)
        config_file = cursor.fetchone()
        
            
        if (config_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            return bz2.decompress(config_file[0]).decode('ascii')
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

cf1 = get_configuration_file(26732) 
print(cf1)

Getting configuration file of cid= 26732
#
# Automatically generated file; DO NOT EDIT.
# Linux/x86 4.13.3 Kernel Configuration
#
CONFIG_64BIT=y
CONFIG_X86_64=y
CONFIG_X86=y
CONFIG_INSTRUCTION_DECODER=y
CONFIG_OUTPUT_FORMAT="elf64-x86-64"
CONFIG_ARCH_DEFCONFIG="arch/x86/configs/x86_64_defconfig"
CONFIG_LOCKDEP_SUPPORT=y
CONFIG_STACKTRACE_SUPPORT=y
CONFIG_MMU=y
CONFIG_ARCH_MMAP_RND_BITS_MIN=28
CONFIG_ARCH_MMAP_RND_BITS_MAX=32
CONFIG_ARCH_MMAP_RND_COMPAT_BITS_MIN=8
CONFIG_ARCH_MMAP_RND_COMPAT_BITS_MAX=16
CONFIG_NEED_DMA_MAP_STATE=y
CONFIG_NEED_SG_DMA_LENGTH=y
CONFIG_GENERIC_ISA_DMA=y
CONFIG_GENERIC_BUG=y
CONFIG_GENERIC_BUG_RELATIVE_POINTERS=y
CONFIG_GENERIC_HWEIGHT=y
CONFIG_ARCH_MAY_HAVE_PC_FDC=y
CONFIG_RWSEM_XCHGADD_ALGORITHM=y
CONFIG_GENERIC_CALIBRATE_DELAY=y
CONFIG_ARCH_HAS_CPU_RELAX=y
CONFIG_ARCH_HAS_CACHE_LINE_SIZE=y
CONFIG_HAVE_SETUP_PER_CPU_AREA=y
CONFIG_NEED_PER_CPU_EMBED_FIRST_CHUNK=y
CONFIG_NEED_PER_CPU_PAGE_FIRST_CHUNK=y
CONFIG_ARCH_HIBERNATION_POSSIBLE=y
CONFIG_ARCH_SUSPEND_P

In [5]:
cf_tiny = get_configuration_file(34458) 
ny = 0
for l in cf_tiny.splitlines():
    if l.endswith("=y"):
        ny = ny + 1
        #print(l)
print("number of active options", ny)

Getting configuration file of cid= 74458
number of active options 240


In [12]:
def get_linux_version(cid):
    print("Getting cid=", cid)
    
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        cursor = socket.cursor()

        query = "SELECT compiled_kernel_version FROM compilations WHERE cid = %d" % cid
        cursor.execute(query)
        l_version = cursor.fetchone()
        
            
        if (l_version is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        return l_version[0]
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database cid = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

v1 = get_linux_version(26732) 
print(v1)
v2 = get_linux_version(36732) 
print(v2)

Getting cid= 26732
4.13.3
Getting cid= 36732
4.15


In [28]:
def _quote(str):
    return '"' + str + '"'

def get_build_failures(linux_version="4.15"):
        
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        cursor = socket.cursor()
        # WHERE compressed_compiled_kernel_size = " + str(linux_version)  
        query = "SELECT COUNT(*) FROM compilations WHERE `compiled_kernel_version` = " + _quote(linux_version) + " AND compiled_kernel_size <= 0"
        cursor.execute(query)
        return cursor.fetchone()
        
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database = " + str(linux_version), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

get_build_failures()


(490,)

In [82]:
import pandas as pd
def get_configs_cid(cid_start, cid_end, linux_version="4.15"):
        
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        cursor = socket.cursor()
        # WHERE compressed_compiled_kernel_size = " + str(linux_version)  
        query = "SELECT config_file FROM compilations WHERE `compiled_kernel_version` = " + _quote(linux_version) + " AND cid >= " + str(cid_start) + " AND " + "cid <= " + str(cid_end)
        cursor.execute(query)
        
        return list(cursor.fetchall())
        
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database = " + str(linux_version), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

def isWhitespace(c):
    return c == ' ' or c == '\t' or c == '\n'

def scanConfig(config):

    props = {}
    cursor = 0
    s = len(config)
    state = 0
    # 0 -> seeking name, comment, or end of file
    # 1 -> skipping comment
    # 2 -> writing name
    # 3 -> writing value
    name = ""
    value = ""
    while cursor < s:
        c = config[cursor]
        if state == 0:
            # Seek comment
            if c == '#':
                state = 1
            # Seek name
            elif not isWhitespace(c):
                state = 2
                name = c
        elif state == 1:
            # Seek end of line
            if c == '\n':
                state = 0
        elif state == 2:
            # Continue name
            if c != '=':
                name += c
            # Seek value
            else:
                state = 3
                value = ""
        elif state == 3:
            # Continue value
            if c != '\n':
                value += c
            else:
                # Validate name + value
                # Remove quotes
                if value[0] == '"' and value[-1] == '"':
                    value = value[1:-1]
                # Set in dict and remove leading "CONFIG_" in name
                props[name[7:]] = value
                state = 0
        cursor += 1
    if state == 2:
        raise ValueError("Incomplete .config file : " + str(props))
    return props

cfs = get_configs_cid(cid_start=30000, cid_end=30500, linux_version="4.15")
print("configs retrieved from the database")
cfi = 0
df_configs = pd.DataFrame()
a_configs = [] # pd.DataFrame()
for cf in cfs:
    config = bz2.decompress(cf[0]).decode('ascii')
    props = scanConfig(config)
    #print(pd.Series(props))
    df_config = pd.Series(props) # DataFrame(props, index=[cfi])
#    a_configs.append(df_config) #append(df_config)
    cfi = cfi + 1
    df_configs = pd.concat([df_configs, df_config], axis=1, sort=False)
    print(cfi)
    #for (k, v) in props.items():
            # values[order[k]] = v
    #        print(k, v)
    # print(props)
    

configs retrieved from the database
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268


In [ ]:
# df_configs = pd.concat(a_configs, axis=0, sort=False)

In [83]:
df_configs.shape

(9852, 492)

In [84]:
df_configs.transpose()

64BIT X86_64 X86 INSTRUCTION_DECODER OUTPUT_FORMAT  \
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
..   ...    ...  ..                 ...           ...   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   
0      y      y   y                   y  elf64-x86-64   

                       ARCH_DEFCONFIG LOCKDEP_SUPPORT STACKTRACE_SUPPORT MMU  \
0   arch/x86/configs/x86_64_defconfig               y                  y   y   
0   arch/x86/configs/x86_64_defconfig               y                  y   y   
0   arch/x86/configs/x86_64_defconfig               y                  y   y   
0   arch/x86/configs/x86_64_defconfig               y                  y   y   
0   arch/x86/configs/x86_64_defconfig               y               